In [1]:
%matplotlib qt5 
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
from scipy.interpolate import interp1d
from fastdtw import fastdtw

In [2]:
"""Config"""
word = "time" 
discart_points_face = True
participant_id = 29302
landmarks = 75 # 75 sin puntos de la cara y con puntos de la cara son 543
data = pd.read_csv("train.csv")
filtered_data = data[data["sign"] == word]
csv_path = "filter_for_word.csv"
filtered_data.to_csv(csv_path, index=False)
participant_ids = filtered_data["participant_id"].unique()

In [3]:
#leer un .parquet aleatorio
path_one_file = filtered_data.iloc[6]["path"]
df = pd.read_parquet(path_one_file)


In [4]:
def process_path_data(path_number, filtered_data):
    """
    Procesa los datos de una ruta específica (los .parquet), extrae y reformatea la información relevante de los puntos (x, y).

    Args:
        path_number (int): El índice de la ruta en el DataFrame filtered_data.
        filtered_data (pandas.DataFrame): Un DataFrame que contiene información sobre las rutas de los archivos de datos.

    Returns:
        numpy.ndarray: Un array de la secuencia reformateada con la forma (num_samples, 543, 2), donde num_samples es el
                       número de fotogramas únicos en los datos.
    """
    path_file = filtered_data.iloc[path_number]["path"]
    data = pd.read_parquet(path_file)
    if discart_points_face == True:
        data = data[data["type"] != "face"]
    cleaned_data = data[["frame", "x", "y"]]
    num_samples = cleaned_data["frame"].nunique()
    cleaned_data = cleaned_data[["x", "y"]].to_numpy()
    seq = cleaned_data.reshape(num_samples, landmarks, 2)
    return seq

In [5]:
def add_descriptors(sequence):
    """
    Añade los descriptores de derivada a una secuencia.

    Args:
        sequence (numpy.ndarray): La secuencia de entrada con la forma (num_frames, landmarks, 2).

    Returns:
        numpy.ndarray: La secuencia con los descriptores de derivada añadidos con la forma (num_frames, landmarks, 4).
    """
    # Calcula la derivada x e y para cada fotograma de la secuencia
    dx = np.gradient(sequence[:, :, 0], axis=0)
    dy = np.gradient(sequence[:, :, 1], axis=0)
    
    # Concatena la secuencia original con las derivadas dx e dy
    sequence_with_descriptors = np.concatenate([sequence, dx[:, :, np.newaxis], dy[:, :, np.newaxis]], axis=2)

    return sequence_with_descriptors


In [6]:
def interpolate_sequence(seq, num_frames):
    """
    Interpola una secuencia única a una longitud específica en la dimensión del tiempo.
    
    Args:
        seq (numpy.ndarray): La secuencia de entrada con la forma (T, landmarks, 2), donde T es el número de fotogramas.
        num_frames (int): El número de fotogramas para la secuencia interpolada.

    Returns:
        numpy.ndarray: La secuencia interpolada con la forma (num_frames, landmarks, 2).
    """
    seq_interp = np.zeros((num_frames, landmarks, 2))
    seq = np.nan_to_num(seq nan=0)
    
    for i in range(landmarks):
        for j in range(2):
            f = interp1d(np.arange(seq.shape[0]), seq[:, i, j], kind='linear')
            seq_interp[:, i, j] = f(np.linspace(0, seq.shape[0] - 1, num_frames))

    return seq_interp

def interpolate_sequences_list(sequences, num_frames):
    """
    Interpola una lista de secuencias a una longitud específica en la dimensión del tiempo.
    
    Args:
        sequences (list): Una lista de secuencias, cada una con la forma (T, 543, 2), donde T es el número de fotogramas.
        num_frames (int): El número de fotogramas para las secuencias interpoladas.

    Returns:
        numpy.ndarray: Un array que contiene las secuencias interpoladas con la forma (num_frames, 543, 2, num_sequences),
                       donde num_sequences es el número de secuencias en la lista de entrada.
    """
    num_sequences = len(sequences)
    interpolated_sequences = np.zeros((num_frames, landmarks, 2, num_sequences))
    
    for idx, seq in enumerate(sequences):
        seq_interp = interpolate_sequence(seq, num_frames)
        interpolated_sequences[:, :, :, idx] = seq_interp
    
    return interpolated_sequences

num_frames = 30 # Número de fotogramas para las secuencias interpoladas
sequences = [] # Lista de secuencias
participant_ids = filtered_data["participant_id"].unique()
for participant_id in participant_ids:
    participant_data = filtered_data[filtered_data["participant_id"] == participant_id]
    for i in range(len(participant_data)):
        seq = process_path_data(i, participant_data)
        sequences.append(seq)

interpolated_sequences = interpolate_sequences_list(sequences, num_frames)
interpolated_sequences = interpolated_sequences.transpose((3,0,1,2)) #reorganizacion

# Añade los descriptores de derivada para cada secuencia interpolada
interpolated_sequences_with_descriptors = []
for seq in interpolated_sequences:
    seq_with_descriptors = add_descriptors(seq)
    interpolated_sequences_with_descriptors.append(seq_with_descriptors)

# Convierte la lista de secuencias interpoladas con descriptores en un array de NumPy
interpolated_sequences_with_descriptors = np.array(interpolated_sequences_with_descriptors)

print(interpolated_sequences_with_descriptors.shape) # (num_sequences, num_frames, landmarks, n-descriptors)

SyntaxError: invalid syntax (296652730.py, line 13)

In [ ]:
""" export interpolated_sequences to npy"""
if discart_points_face is True:
    np.save(f"interpolated_sequences_{word}_75points_with_descriptors.npy", interpolated_sequences_with_descriptors)
    plt.plot(interpolated_sequences_with_descriptors[1, :, 0], interpolated_sequences_with_descriptors[1, :, 1], 'o')
else:
    np.save(f"interpolated_sequences_{word}_543points_with_descriptors.npy", interpolated_sequences_with_descriptors)
    plt.plot(interpolated_sequences_with_descriptors[1, :, 0], interpolated_sequences_with_descriptors[1, :, 1], 'o')


In [ ]:
""" visualizar datos"""

# data_descriptors_rendering = np.load(f"interpolated_sequences_{word}_543points.npy")




' visualizar datos'